In [74]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
from nltk import FreqDist
from matplotlib.font_manager import FontProperties
from IPython.core.interactiveshell import InteractiveShell


pd.set_option('display.max_rows', 100) # 设置最大显示100行
pd.set_option('display.max_columns', 100) # 设置最大显示100列
myfont = FontProperties(fname = "C:\Windows\Fonts\simhei.ttf", size = 14)
sns.set(font = myfont.get_name())
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
InteractiveShell.ast_node_interactivity = "all" # 一个cell显示多输出

In [26]:
cleanedDataPcf = pd.read_csv("CleanedData/pacifier.csv", index_col = 0, parse_dates=['review_date'])

encodedDataPcf = deepcopy(cleanedDataPcf)

cleanedDataMcw = pd.read_csv("CleanedData/microwave.csv", index_col = 0, parse_dates=['review_date'])

encodedDataMcw = deepcopy(cleanedDataMcw)

cleanedDataHdr = pd.read_csv("CleanedData/hair_dryer.csv", index_col = 0, parse_dates=['review_date'])

encodedDataHdr = deepcopy(cleanedDataHdr)

In [27]:

encodedDataPcf = encodedDataPcf[["customer_id", "review_id", 
                                 "product_id", "star_rating", 
                                 "helpful_votes", "total_votes", 
                                 "vine", "verified_purchase", 
                                 "review_headline", "review_body", 
                                "review_date"]]

encodedDataMcw = encodedDataMcw[["customer_id", "review_id", 
                                 "product_id", "star_rating", 
                                 "helpful_votes", "total_votes", 
                                 "vine", "verified_purchase", 
                                 "review_headline", "review_body", 
                                "review_date"]]

encodedDataHdr = encodedDataHdr[["customer_id", "review_id", 
                                 "product_id", "star_rating", 
                                 "helpful_votes", "total_votes", 
                                 "vine", "verified_purchase", 
                                 "review_headline", "review_body", 
                                "review_date"]]

只提取有意义的特征

In [28]:
encodedDataPcf["vine"] = (encodedDataPcf["vine"] == 'Y') | (encodedDataPcf["vine"] == 'y')

encodedDataMcw["vine"] = (encodedDataMcw["vine"] == 'Y') | (encodedDataMcw["vine"] == 'y')

encodedDataHdr["vine"] = (encodedDataHdr["vine"] == 'Y') | (encodedDataHdr["vine"] == 'y')

encodedDataPcf["verified_purchase"] = (encodedDataPcf["verified_purchase"] == 'Y') | (encodedDataPcf["verified_purchase"] == 'y')

encodedDataMcw["verified_purchase"] = (encodedDataMcw["verified_purchase"] == 'Y') | (encodedDataMcw["verified_purchase"] == 'y')

encodedDataHdr["verified_purchase"] = (encodedDataHdr["verified_purchase"] == 'Y') | (encodedDataHdr["verified_purchase"] == 'y')

把逻辑特征转为布尔值

In [29]:
encodedDataPcf["product_id"] = encodedDataPcf["product_id"].str.upper()

encodedDataMcw["product_id"] = encodedDataMcw["product_id"].str.upper()

encodedDataHdr["product_id"] = encodedDataHdr["product_id"].str.upper()

encodedDataPcf["customer_id"] = encodedDataPcf["customer_id"].astype("str")

encodedDataMcw["customer_id"] = encodedDataMcw["customer_id"].astype("str")

encodedDataHdr["customer_id"] = encodedDataHdr["customer_id"].astype("str")

encodedDataPcf["review_id"] = encodedDataPcf["review_id"].str.upper()

encodedDataMcw["review_id"] = encodedDataMcw["review_id"].str.upper()

encodedDataHdr["review_id"] = encodedDataHdr["review_id"].str.upper()

处理Unique Id

In [64]:
def merge_review(df: pd.DataFrame):
    df['review_headline'] = df['review_headline'].apply(str)
    df['review_body'] = df['review_body'].apply(str)
    df['review'] = df['review_headline'] + '. ' + df['review_body']

In [67]:
merge_review(encodedDataPcf)

merge_review(encodedDataMcw)

merge_review(encodedDataHdr)

In [69]:
encodedDataPcf.drop(['review_headline', 'review_body'], axis=1)

encodedDataMcw.drop(['review_headline', 'review_body'], axis=1)

encodedDataHdr.drop(['review_headline', 'review_body'], axis=1)

,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date,review_headline_score,review
0,40626522,R1A3ZUBR8TSAKY,B00793CZAE,5,0,0,False,True,2015-08-31,0.818,Love this!. Perfect match for the Gund Huggybu...
1,16290022,RLJNYBK4FGBYX,B003PCYMP4,5,0,0,False,True,2015-08-31,0.895,Love 💕. My little girl love this paci contrapt...
2,10216509,R26QCW75C4JDOK,B003CK3LDI,5,0,0,False,True,2015-08-31,0.000,Five Stars. My son loves this one and will onl...
3,114040,R2E7N0TVLUHUDR,B003CK3LDI,5,0,0,False,True,2015-08-31,0.000,Five Stars. Perfect
4,27971579,R1SO9VMCIGZX3U,B003PCYMP4,5,0,0,False,True,2015-08-31,0.000,Five Stars. Amazing addition to the nursery!
...,...,...,...,...,...,...,...,...,...,...,...
11046,21573136,R3BWX9SX2KS651,B00007C65S,2,0,0,False,False,2004-05-24,0.000,Not for bottle liners. We have been using the ...
11047,19606706,R38BZDRPIX74RX,B00007C65S,4,1,1,False,False,2004-04-04,0.492,good enough for me. This isn't the greatest pr...
11048,25764155,R1XNX6XGDIHJ7G,B00007C65S,5,0,0,False,False,2004-04-04,0.692,I love it!. I love this bottle warmer. After ...
11049,28162301,R1TW1Z1I35H29N,B00007C65S,1,2,2,False,False,2003-12-02,0.615,save your money. I finally broke down and open...


,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date,review
0,21879631,RY52KZABZK8QF,B0052G14E8,1,0,0,False,True,2015-08-31,Do not recommend it. go use your money for som...
1,14964566,R3GCOEV4HYZG2I,B0055UBB4O,5,0,0,False,True,2015-08-31,Looks as good as the picture. Didn't use it lo...
2,13230389,R1V2OPPNL0QGCE,B0052G14E8,4,0,0,False,True,2015-08-31,"Four Stars. Very nice microwave, great price"
3,43655888,R9Q0QDTLKV567,B004ZU09QQ,3,0,1,False,True,2015-08-31,"quiet. Quiet, but does not seem like 1000 watt..."
4,117794,R3DL7HYC3QTWNI,B005GSZB7I,4,0,0,False,True,2015-08-31,Four Stars. It's bigger than I thought.
...,...,...,...,...,...,...,...,...,...,...
1598,49010899,R2OXHMMI830KJ3,B00009V3X6,2,12,14,False,False,2004-09-13,"A great, sleek oven... if you can get an undam..."
1599,40308862,RY9INWIK8MAL3,B00012ORT2,3,20,53,False,False,2004-07-20,What Happened to Sharp Quality?. I do not have...
1600,36386173,RXOAWGI9Z98QY,B00009V3WZ,4,19,19,False,True,2004-06-25,"Great microwave, little tough to mount. Its a ..."
1601,18471248,RVNWLGVDWE20J,B00009V3X8,3,6,6,False,True,2004-06-21,Sharp could be Sharper. UPDATE: Yes i too had ...


,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_date,review
0,34678741,R9T1FE2ZX2X04,B003V264WW,5,0,0,False,True,2015-08-31,Works great. Works great!
1,11599505,RE36JAD5V53PO,B0009XH6V4,4,0,0,False,True,2015-08-31,I love travel blow dryers because they are eas...
2,2282190,RIDHM8B7SCCV3,B0007NZPY6,5,0,1,False,True,2015-08-31,Five Stars. Love this dryer!
3,43669858,R14QGWPCHU9LSE,B00BB8ZIW0,5,0,0,False,True,2015-08-31,Five Stars. styling hair in style
4,107098,R35BHQJHXXJD59,B003V264WW,4,0,0,False,False,2015-08-31,I think's great. The cord length is perfect. I...
...,...,...,...,...,...,...,...,...,...,...
11106,35599734,R2O50YNP83CG34,B000065DJY,5,14,18,False,False,2002-08-21,"great hair!. I just purchased this dryer, and ..."
11107,43977044,R2JQPUYU65C4QD,B00005O0MZ,1,26,32,False,False,2002-08-13,Disappointment with dryer. I purchased it beca...
11108,37733836,R3GO6L5PWBS0IW,B00005O0MZ,5,4,4,False,True,2002-07-13,The best dryer!!!. I love this dryer not only ...
11109,38947355,R3JMGN42OJCL97,B00005O0MZ,5,78,87,False,False,2002-04-20,Excellent for fine/limp hair. This hairdryer f...


In [73]:
def freq_words(x, terms = 30):
    all_words = ' '.join([text for text in x])
    all_words = all_words.split()

    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})

    # selecting top 20 most frequent words
    d = words_df.nlargest(columns="count", n = terms) 
    plt.figure()
    ax = sns.barplot(data=d, x="count" , y = "word")
    ax.set(ylabel = 'word')

In [70]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def get_emotional_intensity(sentence: str) ->float:
    analyzer = SentimentIntensityAnalyzer()
    score_dict = analyzer.polarity_scores(sentence)

    return score_dict['compound']

In [71]:
encodedDataPcf["review_score"] = encodedDataPcf["review_body"].map(get_emotional_intensity)

encodedDataMcw["review_score"] = encodedDataMcw["review_body"].map(get_emotional_intensity)

encodedDataHdr["review_score"] = encodedDataHdr["review_body"].map(get_emotional_intensity)

KeyboardInterrupt: 